In [1]:
using DataFrames, CSV, DataArrays, Convex, SCS

In [2]:
GenData = readtable("gen_3.csv")
BusData = readtable("bus_3.csv")
BranchData = readtable("branch_3.csv")
Gen_cost = readtable("gencost_3.csv")

,x2,startup,shutdown,n,c2,c1,c0
1,2,1500,0,3,0.11,5.0,0
2,2,2000,0,3,0.085,1.2,0
3,2,3000,0,3,0.1225,1.0,0


In [3]:
# Function for the admittance matrix of the network

function ybus(BusData, BranchData)
    
    # Y = 1/Z
    BranchData[:z] = BranchData[:r] + im*BranchData[:x]
    BranchData[:y] = 1 ./BranchData[:z]
    
    # Numbers of lines and buses
    NLines = size(BranchData,1)
    NBuses = size(BusData,1)
    
    # Creating Admittance matrix Y
    Y = zeros(Complex, NBuses, NBuses)
    Ynm = zeros(Complex, NBuses, NBuses)
    
    B_lm = zeros(Complex, NBuses, NBuses)
    y_lm = zeros(Complex, NBuses, NBuses)
    
    for l in 1:NBuses
        B_lm[BranchData[l, :fbus], BranchData[l, :tbus]] = 0.5*im*BranchData[l, :b]
        B_lm[BranchData[l, :tbus], BranchData[l, :fbus]] = 0.5*im*BranchData[l, :b]
        y_lm[BranchData[l, :fbus], BranchData[l, :tbus]] = BranchData[l, :y]
        y_lm[BranchData[l, :tbus], BranchData[l, :fbus]] = BranchData[l, :y]
    end

    
       # Non-diagonal Y
    for l in 1:NLines
        Y[BranchData[l, :fbus], BranchData[l, :tbus]] = -BranchData[l, :y]
        Y[BranchData[l, :tbus], BranchData[l, :fbus]] = -BranchData[l, :y]
    end

    # Diagonal elements
    for i = 1:NBuses
        for j = 1:NLines
            if (i ==BranchData[j, :fbus])|| (i==BranchData[j, :tbus])
                Y[i,i] += BranchData[j, :y] + 0.5*im*BranchData[j, :b]
            end
        end
    end
    
    return Y, B_lm, y_lm
end

ybus (generic function with 1 method)

In [4]:
Ybus, B_lm, y_lm = ybus(BusData, BranchData)

(Complex[0.218996-2.32906im -0.0517392+1.1087im -0.167256+1.59537im; -0.0517392+1.1087im 0.0961343-1.94055im -0.0443951+1.33185im; -0.167256+1.59537im -0.0443951+1.33185im 0.211651-2.35222im], Complex[0+0im 0.0+0.15im 0.0+0.225im; 0.0+0.15im 0+0im 0.0+0.35im; 0.0+0.225im 0.0+0.35im 0+0im], Complex[0+0im 0.0517392-1.1087im 0.167256-1.59537im; 0.0517392-1.1087im 0+0im 0.0443951-1.33185im; 0.167256-1.59537im 0.0443951-1.33185im 0+0im])

In [5]:
function Ylm(B_lm, y_lm, l, m)
    e_l = zeros(NBuses,1)
    e_m = zeros(NBuses,1)
    e_l[l], e_m[m] = 1, 1
    
    Y_lm = (B_lm[l,m]+y_lm[l,m])*e_l*transpose(e_m)
end

Ylm (generic function with 1 method)

In [6]:
NBuses = size(BusData,1)

3

In [7]:
function Y_func(i, Ybus, NBuses)
    e_i = zeros(NBuses,1)
    e_i[i] = 1
    YY = e_i*transpose(e_i)*Ybus
    return YY
end
Y_func(1, Ybus, NBuses)

3×3 Array{Number,2}:
 0.218996-2.32906im  -0.0517392+1.1087im  -0.167256+1.59537im
      0.0+0.0im             0.0+0.0im           0.0+0.0im    
      0.0+0.0im             0.0+0.0im           0.0+0.0im    

In [8]:
Y_func_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_func_array[j] = Y_func(j, Ybus, NBuses)
end
Y_func_array[1] 

3×3 Array{Number,2}:
 0.218996-2.32906im  -0.0517392+1.1087im  -0.167256+1.59537im
      0.0+0.0im             0.0+0.0im           0.0+0.0im    
      0.0+0.0im             0.0+0.0im           0.0+0.0im    

In [9]:
function Y_i(i, YY, NBuses)
    e_i = zeros(NBuses,1)
    e_i[i] = 1
    
    Yi = 0.5 *[[real(YY + transpose(YY)) imag(transpose(YY) - YY)]; [imag(YY - transpose(YY)) real(YY + transpose(YY))]]
    
    Yi_i = -0.5 *[[imag(YY + transpose(YY)) real(YY - transpose(YY))]; [real(transpose(YY) - YY) imag(YY + transpose(YY))]]

    M = [[e_i*transpose(e_i) zeros(NBuses, NBuses)]; [zeros(NBuses, NBuses) e_i*transpose(e_i)]]
    
    return Yi, Yi_i, M
end

Y_i (generic function with 1 method)

In [10]:
Y_i_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_i_array[j] = Y_i(j, Y_func_array[j], NBuses)
end
size(Y_i_array)

(3, 1)

In [11]:
Y_k_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_k_array[j] = Y_i_array[j][1]
end
size(Y_k_array[1])

(6, 6)

In [12]:
Y_ii_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_ii_array[j] = Y_i_array[j][2]
end
Y_ii_array[1]

6×6 Array{Float64,2}:
  2.32906    -0.554348   -0.797684   -0.0         0.0258696   0.0836282
 -0.554348   -0.0        -0.0        -0.0258696  -0.0        -0.0      
 -0.797684   -0.0        -0.0        -0.0836282  -0.0        -0.0      
 -0.0        -0.0258696  -0.0836282   2.32906    -0.554348   -0.797684 
  0.0258696  -0.0        -0.0        -0.554348   -0.0        -0.0      
  0.0836282  -0.0        -0.0        -0.797684   -0.0        -0.0      

In [13]:
Y_M_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_M_array[j] = Y_i_array[j][3]
end
Y_M_array[1]

6×6 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [14]:
function YY_lm(B_lm, y_lm, l, m)
    Y_lm = Ylm(B_lm, y_lm, l, m)
    Y_lm_first = 0.5 *[[real(Y_lm + transpose(Y_lm)) imag(transpose(Y_lm) - Y_lm)]; [imag(Y_lm - transpose(Y_lm)) real(Y_lm + transpose(Y_lm))]]
    
    Y_lm_second = -0.5 *[[imag(Y_lm + transpose(Y_lm)) real(Y_lm - transpose(Y_lm))]; [real(transpose(Y_lm) - Y_lm) imag(Y_lm + transpose(Y_lm))]]
    
    return Y_lm_first, Y_lm_second
end

YY_lm (generic function with 1 method)

In [15]:
Y_lm_array = Matrix(NBuses,NBuses)
Y_lm_conj_array = Matrix(NBuses,NBuses)
for l in 1:NBuses
    for m in 1:NBuses
        if l != m
            Y_lm_array[l,m], Y_lm_conj_array[l, m] = YY_lm(B_lm, y_lm, l, m)
        else
            Y_lm_array[l,m], Y_lm_conj_array[l, m] = zeros(NBuses*2, NBuses*2), zeros(NBuses*2, NBuses*2)
        end
    end
end
Y_lm_array[1,2]

6×6 Array{Float64,2}:
 0.0         0.0258696  0.0   0.0        0.479348   0.0
 0.0258696   0.0        0.0  -0.479348   0.0        0.0
 0.0         0.0        0.0   0.0        0.0        0.0
 0.0        -0.479348   0.0   0.0        0.0258696  0.0
 0.479348    0.0        0.0   0.0258696  0.0        0.0
 0.0         0.0        0.0   0.0        0.0        0.0

In [17]:
# Solver 

In [18]:
# Define generators - it is a bus of type 2 or 3 (type 3 is a slack bus)
Gen_indices = Int64[]
for i in 1:NBuses
     if (BusData[i,:_type] == 2)||(BusData[i,:_type] == 3)
        push!(Gen_indices, BusData[i,:bus_i])
     end
end
NI =length(Gen_indices)

#NBuses=max(maximum(BranchData[:,:fbus]),maximum(BranchData[:,:tbus]));
NLines=size(BranchData,1);
NGen=NI;
Buses=1:NBuses;
Lines=1:NLines;
Gen=1:NGen;

In [19]:
# limints in branch powerflow
S_line = zeros(NBuses,NBuses)
S_line = [0 1000 1000;
          1000 0 60;
          1000 60 0;]
S_line[1,2]

1000

In [35]:
i = 1
Gen_cost[i, :c2]

0.11

In [30]:
W = Semidefinite(NBuses*2, NBuses*2)
#@variable(mm, AA[1:3, 1:3], SDP)
#@variable(mm, BB[1:2, 1:2], SDP)
problem = minimize(sum(Gen_cost[i, :c2]*(trace(Y_k_array[i]*W) + BusData[i, :Pd])^2 + Gen_cost[i, :c1]*(trace(Y_k_array[i]*W) + BusData[i, :Pd]) + Gen_cost[i, :c0] for i in Gen_indices))


problem.constraints = [([-(S_line[m,l])^2 trace(Y_lm_array[l,m]*W) trace(Y_lm_conj_array[l,m]*W);
                                                     trace(Y_lm_array[l,m]*W) -1 0;
                                                     trace(Y_lm_conj_array[l,m]*W) 0 -1]   in :SDP) for l in NBuses,  m in NBuses]

#problem.constraints = [([-(S_line[m,l])^2 trace(Y_lm_array[l,m]*W) trace(Y_lm_conj_array[l,m]*W);
#                                                     trace(Y_lm_array[l,m]*W) -1 0;
#                                                     trace(Y_lm_conj_array[l,m]*W) 0 -1]   in :SDP) for l=1,  m=1]
    

problem.constraints += [([Gen_cost[i, :c2]*(trace(Y_k_array[i]*W)+BusData[i, :Pd])^2 -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]);
                                  -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]) 1] in :SDP) for i in NBuses]

problem.constraints += [trace(Y_k_array[i]*W) >= GenData[i, :Pmin] - BusData[i, :Pd] for i = 1:NBuses]
problem.constraints += [trace(Y_k_array[i]*W) <= GenData[i, :Pmax] - BusData[i, :Pd] for i = 1:NBuses]
problem.constraints += [trace(Y_ii_array[i]*W) >=GenData[i, :Qmin] - BusData[i, :Qd] for i = 1:NBuses]
problem.constraints += [trace(Y_ii_array[i]*W) <= GenData[i, :Qmax] - BusData[i, :Qd] for i = 1:NBuses]
problem.constraints += [trace(Y_M_array[i]*W) <= BusData[i, :Vmax]^2 for i = 1:NBuses]
problem.constraints += [trace(Y_M_array[i]*W) >= BusData[i, :Vmin]^2 for i = 1:NBuses]

                
solve!(problem, SCSSolver())
W.value

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 412, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 41, constraints m = 84
Cones:	primal zero / dual free vars: 20
	linear vars: 22
	soc vars: 12, soc blks: 4
	sd vars: 30, sd blks: 3
Setup time: 8.00e-05s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.27e+20  4.48e+20  1.00e+00 -1.27e+23  1.38e+23  8.42e+22  2.02e-04 
    80| 9.50e+00  1.53e+02  4.28e-03  1.12e+11  1.13e+11  9.65e+08  6.70e-03 
------------

6×6 Array{Float64,2}:
 NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN

In [31]:
W = Semidefinite(NBuses*2, NBuses*2)
#@variable(mm, AA[1:3, 1:3], SDP)
#@variable(mm, BB[1:2, 1:2], SDP)
#problem = minimize(sum(Gen_cost[i, :c2]*(trace(Y_k_array[i]*W) + BusData[i, :Pd])^2 + Gen_cost[i, :c1]*(trace(Y_k_array[i]*W) + BusData[i, :Pd]) + Gen_cost[i, :c0] for i in Gen_indices))


constraints = [([[-(S_line[m,l])^2 trace(Y_lm_array[l,m]*W) trace(Y_lm_conj_array[l,m]*W)];
                 [trace(Y_lm_array[l,m]*W) -1 0];
                 [trace(Y_lm_conj_array[l,m]*W) 0 -1]] in :SDP) for l = 1:NBuses,  m = 1:NBuses]

constraints +=                [([Gen_cost[i, :c2]*(trace(Y_k_array[i]*W)+BusData[i, :Pd])^2 -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]);
                                                  -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]) 1] in :SDP) for i = 1:NBuses]

constraints +=                [trace(Y_k_array[i]*W) >= GenData[i, :Pmin] - BusData[i, :Pd] for i = 1:NBuses]
constraints +=                [trace(Y_k_array[i]*W) <= GenData[i, :Pmax] - BusData[i, :Pd] for i = 1:NBuses]
constraints +=                [trace(Y_ii_array[i]*W) >=GenData[i, :Qmin] - BusData[i, :Qd] for i = 1:NBuses]
constraints +=                [trace(Y_ii_array[i]*W) <= GenData[i, :Qmax] - BusData[i, :Qd] for i = 1:NBuses]
constraints +=                [trace(Y_M_array[i]*W) <= BusData[i, :Vmax]^2 for i = 1:NBuses]
constraints +=                [trace(Y_M_array[i]*W) >= BusData[i, :Vmin]^2 for i = 1:NBuses]
                                    
p = minimize((sum(Gen_cost[i, :c2]*(trace(Y_k_array[i]*W) + BusData[i, :Pd])^2 + Gen_cost[i, :c1]*(trace(Y_k_array[i]*W) + BusData[i, :Pd]) + Gen_cost[i, :c0] for i in Gen_indices)),constraints)

Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

subject to
Constraint:
sdp constraint
expression: AbstractExpr with
head: transpose
size: (3, 3)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


		Constraint:
sdp constraint
expression: AbstractExpr with
head: transpose
size: (3, 3)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


		Constraint:
sdp constraint
expression: AbstractExpr with
head: transpose
size: (3, 3)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


		Constraint:
sdp constraint
expression: AbstractExpr with
head: transpose
size: (3, 3)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


		Constraint:
sdp constraint
expression: AbstractExpr with
head: transpose
size: (3, 3)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


		Constraint:
sdp constraint
expression: AbstractExpr with
head: transpose
size: (3, 3)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


		Constraint:
sdp constraint
